In [1]:
from hpo import gbm_hpo, nn_hpo, ae_hpo
from dotenv import load_dotenv
import os
import numerapi

def credentials():
    dotenv_path = 'num_config.env'
    load_dotenv(dotenv_path=dotenv_path)
    pub_id = os.getenv('PUBLIC_ID')
    priv_key = os.getenv('PRIVATE_KEY')
    latest_round = os.getenv('LATEST_ROUND')
    #pub_id = "C4Q5XUHAH3MSMAHHHRHSQWHG2SUW5Q54"
    #priv_key = "QSBJY72HUPPDJBP3UF7JPFHRK4CV3ITWOEWQNWXD44RUTVORXQCB5BYIQK7I4CMD"
    #latest_round = 156
    return {'PUBLIC_ID': pub_id, 'PRIVATE_KEY': priv_key, 'LATEST_ROUND': latest_round}


def download_data(api: numerapi.NumerAPI, keys):
    if int(keys['LATEST_ROUND']) == api.get_current_round():
        return int(keys['LATEST_ROUND'])
    else:
        LATEST_ROUND = api.get_current_round()
        api.download_current_dataset('./data')
        return LATEST_ROUND


def update_env_file(env_vars):
    with open('num_config.env', 'w') as f:
        f.write(f'LATEST_ROUND={env_vars["LATEST_ROUND"]}\n')
        f.write(f'PUBLIC_ID={env_vars["PUBLIC_ID"]}\n')
        f.write(f'PRIVATE_KEY={env_vars["PRIVATE_KEY"]}\n')

def create_preds():
    pass

keys = credentials()
numapi = numerapi.NumerAPI(
        verbosity='INFO', public_id=keys['PUBLIC_ID'], secret_key=keys['PRIVATE_KEY'])
keys['LATEST_ROUND'] = download_data(numapi, keys)
update_env_file(keys)
#gbm_hpo.main()
#ae_hpo.main()
#print("nn_hpo...")
# nn_hpo.main(train_ae=False)


In [2]:
import copy
import gc
from data_loading import utils, purged_group_time_series
from data_loading.utils import load_data, preprocess_data, FinData, weighted_mean, seed_everything, calc_data_mean, \
    create_dataloaders
from models import resnet, lightning_nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch.nn as nn
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl
import numpy as np
import torch
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import lightgbm as lgb
import optuna
import joblib
from models.SupervisedAutoEncoder import SupAE, create_hidden_rep
import os
from data_loading import purged_group_time_series as pgs


In [4]:
#p_ae = {'dim_1': 675, 'dim_2': 400, 'dim_3': 224, 'hidden': 162,
#         'activation': nn.ReLU, 'dropout': 0.2916447561918717, 'lr': 0.030272591341587315,
#         'recon_loss_factor': 0.4447516076774931, 'batch_size': 1252, 'loss_sup_ae': nn.MSELoss,
#         'loss_recon': nn.MSELoss,
#         'embedding': True,
#         'input_size':310,'output_size':1}

#p_res = {'dim_1': 843, 'dim_2': 2637, 'dim_3': 1618, 'dim_4': 880, 'dim_5': 220, 'activation': nn.LeakyReLU, 'dropout': 0.453246718032545, 'lr': 0.04565788979670108, 'batch_size': 10836,'loss':nn.MSELoss,'embedding':True,'input_size':310,'output_size':1,'hidden_len':p_ae['hidden']}
#model_res = resnet.ResNet(input_size=p_res['input_size'],output_size=p_res['output_size'],params=p_res)
#model_res.load_state_dict(torch.load('./saved_models/trained/ResNet_state_dict.pth'))
#model_ae = utils.load_model(path='./saved_models/trained/trained_ae.pth',p=p_ae,pl_lightning=False,model=SupAE)
p_lgbm = 	{'learning_rate': 0.010758658263835987, 'max_leaves': 80, 'bagging_fraction': 0.4927572426577223, 'bagging_freq': 10, 'feature_fraction': 0.4023243616563433, 'min_data_in_leaf': 686, 'lambda_l1': 0.04534825564399065, 'lambda_l2': 0.0409642610305239, 'boosting': 'gbdt', 'objective': 'regression',
         'verbose':          1,
         'n_jobs':           4,
         'metric':           'mse'}
data = utils.load_data('data/', mode='train')
data, target, features, era = utils.preprocess_data(data, nn=True)
data_dict = {'data':     data, 'target': target,
                 'features': features, 'era': era}
scores = []
sizes = []
models_lgbm = []
# gts = GroupTimeSeriesSplit()
gts = pgs.PurgedGroupTimeSeriesSplit(n_splits=5, group_gap=10)
seed_everything(0)
for i, (tr_idx, val_idx) in enumerate(gts.split(data_dict['data'], groups=data_dict['era'])):
    sizes.append(len(tr_idx) + len(val_idx))
    x_tr, x_val = data_dict['data'][tr_idx], data_dict['data'][val_idx]
    y_tr, y_val = data_dict['target'][tr_idx], data_dict['target'][val_idx]
    train = lgb.Dataset(x_tr, label=y_tr)
    val = lgb.Dataset(x_val, label=y_val)
    clf = lgb.train(p_lgbm, train, 500, valid_sets=[val], early_stopping_rounds=50, verbose_eval=True)
    preds = clf.predict(x_val)
    score = mean_squared_error(y_val, preds)
    scores.append(score)
    models_lgbm.append(clf)
    del clf, preds, train, val, x_tr, x_val, y_tr, y_val, score
    rubbish = gc.collect()
#model_lgbm = clf
models_dict = {'lgb':[models_lgbm, p_lgbm]}
#models_dict = {'ResNet':[model_res,p_res],'ae':[model_ae,p_ae]}
#models_dict = {'ae':[model_ae,p_ae]}


alid_0's l2: 0.0497545
[189]	valid_0's l2: 0.0497545
[190]	valid_0's l2: 0.0497545
[191]	valid_0's l2: 0.0497542
[192]	valid_0's l2: 0.0497541
[193]	valid_0's l2: 0.049754
[194]	valid_0's l2: 0.049754
[195]	valid_0's l2: 0.049754
[196]	valid_0's l2: 0.0497538
[197]	valid_0's l2: 0.0497537
[198]	valid_0's l2: 0.0497534
[199]	valid_0's l2: 0.0497533
[200]	valid_0's l2: 0.0497533
[201]	valid_0's l2: 0.049753
[202]	valid_0's l2: 0.0497526
[203]	valid_0's l2: 0.0497527
[204]	valid_0's l2: 0.0497523
[205]	valid_0's l2: 0.0497522
[206]	valid_0's l2: 0.0497521
[207]	valid_0's l2: 0.0497526
[208]	valid_0's l2: 0.0497523
[209]	valid_0's l2: 0.0497525
[210]	valid_0's l2: 0.0497524
[211]	valid_0's l2: 0.0497522
[212]	valid_0's l2: 0.0497518
[213]	valid_0's l2: 0.0497516
[214]	valid_0's l2: 0.0497519
[215]	valid_0's l2: 0.0497518
[216]	valid_0's l2: 0.0497518
[217]	valid_0's l2: 0.0497519
[218]	valid_0's l2: 0.0497518
[219]	valid_0's l2: 0.0497518
[220]	valid_0's l2: 0.0497513
[221]	valid_0's l2: 0

In [5]:
from logging import root
from models.SupervisedAutoEncoder import create_hidden_rep
from operator import mod
import os
import random
import re
from typing import List, Tuple
import dotenv
import datatable as dt
from dotenv.main import load_dotenv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from torch.utils.data import Dataset, Subset, BatchSampler, SequentialSampler, DataLoader

root_dir = './data'
test_files_path, test_files_exist = utils.check_test_files(root_dir)
#df_out = pd.DataFrame()
if test_files_exist:
    test_files = os.listdir(test_files_path)
for idx, file in enumerate(test_files):
    df = utils.load_data(root_dir='./data', mode='test',
                    overide=f'{test_files_path}/{file}')
    df['target'] = 0
    data, target, features, era = utils.preprocess_data(data=df, ordinal=True)
    t_idx = np.arange(start=0, stop=len(era), step=1).tolist()
    data_dict = data_dict = {'data': data, 'target': target,
                                'features': features, 'era': era}
    models_lgb=models_lgbm
    x_val = data_dict['data']
    preds = []
    for model in models_lgb:
        preds.append(model.predict(x_val))
    predictions_lgb = np.mean(preds,axis=0)
    df['prediction_lgb'] = predictions_lgb
    df = df[['id', 'prediction_lgb']]
    pred_path = f'{utils.get_data_path(root_dir)}/predictions/{era[0]}'
    df.to_csv(f'{pred_path}.csv')
   


In [ ]:
import importlib
importlib.reload(utils)

#models_lgb = models['lgb'][0]
#p_lgb = models['lgb'][1]
models_lgb=models_lgbm
x_val = data_dict['data']
preds = []
for model in models_lgb:
    preds.append(model.predict(x_val))
predictions_lgb = np.mean(preds,axis=0)
#df['prediction_lgb'] = pd.DataFrame.from_dict({'era':era,'preds':predictions_lgb,'target':target.T})
#df['prediction_lgb'] = 
#df = df[['id', 'prediction_lgb']]

#utils.create_predictions(models=models_dict)

In [ ]:
df1 = pd.DataFrame.from_dict({'era':era,'preds':predictions_lgb,'target':target.T})
#df['prediction_lgb'] = pd.DataFrame.from_dict({'era':era,'preds':predictions_lgb,'target':target.T})



In [ ]:
df['prediction_lgb'] =  predictions_lgb
df = df[['id', 'prediction_lgb']]

In [ ]:
utils.create_prediction_file()

In [28]:
pred_path = f"{utils.get_data_path(root_dir='./data')}/predictions/"

In [30]:
df = utils.load_data(root_dir='./data/',mode='test')
df['era'][df['era'] == 'eraX'] = 'era999'

In [50]:
val_eras = df['era'].unique().tolist()
#val_eras[335]='era999'

In [ ]:
val_eras = df[df['data_type']=='validation']['era'].unique().tolist()

In [22]:
#eras = os.listdir(pred_path)

In [45]:
eras = [f"era{era.replace('.csv','')}" for era in val_eras]

In [46]:
eras = [era for era in eras if era in eras]

In [47]:
eras =[era.replace('era','') for era in eras]

In [48]:
eras = [f'{era}.csv' for era in eras]

In [51]:
import importlib
importlib.reload(utils)

<module 'data_loading.utils' from 'c:\\Users\\e28898\\Documents\\GitHub\\Numerai\\data_loading\\utils.py'>

In [52]:
df_preds = utils.create_prediction_file(eras=eras)
df_example = dt.fread('.\data\numerai_dataset_261\example_predictions.csv').to_pandas()

In [54]:
df_example = dt.fread('./data/numerai_dataset_261/example_predictions.csv').to_pandas()

In [55]:
df_back= df_preds 

In [60]:
df_p=df_preds.set_index('id')

In [61]:
df_e=df_example.set_index('id')

In [62]:
df_p=df_p.reindex_like(df_e)

In [64]:
df_p_r=df_p.reset_index()

In [66]:
df_p[["prediction"]].to_csv('pred.csv')

In [21]:
numapi.upload_predictions("C:/Users/e28898/Documents/GitHub/Numerai/data/numerai_dataset_261/predictions/predictions.csv", tournament=numapi.get_current_round())

2021-04-28 20:57:48,738 INFO numerapi.base_api: uploading predictions...
2021-04-28 20:57:50,835 ERROR numerapi.base_api: invalid tournament


ValueError: invalid tournament

In [ ]:
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]


# convenience method for scoring
def scoring(df):
    return correlation(df['prediction'], df['target'])


# Payout is just the score cliped at +/-25%
def payout(scores):
    return scores.clip(lower=-0.25, upper=0.25)

In [ ]:
df = df.merge(df_preds, on = 'id')

In [ ]:
scores = df.groupby('era').apply(scoring)
np.mean(scores)

In [ ]:
df_preds

In [ ]:
df = utils.load_data(root_dir='./data/',mode='test')
df = df[df['data_type'] == 'validation']


In [ ]:
df_test = df

In [ ]:
df = utils.load_data(root_dir='./data/',mode='train')

In [ ]:
df['era'][df['era'] == 'eraX'] = 'era999'

In [ ]:
data,target,features,era = utils.preprocess_data(df,ordinal=True)
data_test,target_test,features_test,era_test = utils.preprocess_data(df_test,ordinal=True)
"""
data_ = copy.deepcopy(data)
target_ = copy.deepcopy(target)
oe = OrdinalEncoder()
data = oe.fit_transform(data)
target=oe.fit_transform(target_.values.reshape(-1,1)).reshape(-1)
"""

In [ ]:
del df ,df_test

In [ ]:
## Autoencodr training
data = np.concatenate([data,data_test],0)

In [ ]:
target = np.concatenate([target,target_test],0)
era = np.concatenate([era,era_test],0)

In [ ]:
#t_idx = np.arange(start=0,stop=len(era),step=1).tolist()
t_idx =np.where(era <121)[0].tolist()
v_idx =np.where(era >=121)[0].tolist()


In [ ]:
p = joblib.load('./hpo/params/SupAEnn_hpo_2021-04-05.pkl')

In [ ]:
p = p.best_params

In [ ]:
p['activation'] = nn.ReLU
p['loss_sup_ae']= nn.MSELoss
p['loss_recon']= nn.MSELoss
p['embedding']=True

In [ ]:
dataset = utils.FinData(data = data,target=target,era=era)
dataloaders = utils.create_dataloaders(dataset,indexes={'train':t_idx,'val':v_idx},batch_size=p['batch_size'])
p['input_size'] = len(features)
p['output_size'] = 1
model = SupAE(params=p)
es = EarlyStopping(monitor='val_sup_loss',patience=10,min_delta=0.0005,mode='min')
trainer = pl.Trainer(max_epochs=100,gpus=1,callbacks=[es])

In [ ]:
batch = next(iter(dataloaders['val']))
batch['era']

In [ ]:
trainer.fit(model,train_dataloader = dataloaders['train'],val_dataloaders=dataloaders['val'])
torch.save(model.state_dict(),f'./saved_models/trained/ae_state_dict.pth')

In [ ]:
p_xgb = joblib.load('./hpo/params/xgb_hpo_2021-04-05.pkl').best_params
p_lgb = joblib.load('./hpo/params/lgb_hpo_2021-04-05.pkl').best_params
p_lgb['boosting']= 'gbdt'
p_lgb['max_depth']= 50
p_lgb['objective']= 'regression'
p_lgb['metric']= 'mse'
p_lgb['n_jobs']=-1

In [ ]:
data_dict = data_dict = {'data': data, 'target': target,
                                 'features': features, 'era': era}

In [ ]:
x_tr, x_val = data_dict['data'][t_idx], data_dict['data'][v_idx]
y_tr, y_val = data_dict['target'][t_idx],data_dict['target'][v_idx]
d_tr = lgb.Dataset(x_tr,label=y_tr)
d_val = lgb.Dataset(x_val,label=y_val)
clf = lgb.train(p_lgb,d_tr,1000,valid_sets=[d_val],early_stopping_rounds=50,verbose_eval=True)
clf.save_model(f'./saved_models/trained/lgb.bin')

In [ ]:
data_dict = {'data': data, 'target': target,
                 'features': features, 'era': era}

In [ ]:
p_ae['input_size'] = len(features)
p_ae['output_size'] = 1
model = utils.load_model('./saved_models/trained/trained_ae.pth',
                    p=p_ae, pl_lightning=False, model=SupAE)


In [ ]:

models_dict = {'lgb':[clf,p_lgb]}

In [ ]:
ae_output = create_hidden_rep(model, data_dict)
data_dict['hidden'],ae_preds = ae_output['hidden'], ae_output['preds']
data_dict['hidden_true'] = True
p['hidden_len'] = data_dict['hidden'].shape[-1]

In [ ]:
p

In [ ]:
dataset = utils.FinData(
            data=data_dict['data'], target=data_dict['target'], era=data_dict['era'], hidden=data_dict.get('hidden', None))
dataloaders = utils.create_dataloaders(
            dataset, indexes={'train': t_idx}, batch_size=p['batch_size'])

In [ ]:
p_res = {'dim_1': 843, 'dim_2': 2637, 'dim_3': 1618, 'dim_4': 880, 'dim_5': 220, 'activation': nn.LeakyReLU, 'dropout': 0.453246718032545, 'lr': 0.04565788979670108, 'batch_size': 10836,'loss':nn.MSELoss,'embedding':True,'input_size':310,'output_size':1,'hidden_len':p_ae['hidden']}

In [ ]:
model = resnet.ResNet(input_size = p['input_size'],output_size=p['output_size'],params=p)
es = EarlyStopping(monitor='val_mse',patience=10,min_delta=0.0005,mode='min')
trainer = pl.Trainer(max_epochs=100,gpus=1,callbacks=[es])
trainer.fit(model,train_dataloader=dataloaders['train'],val_dataloaders=dataloaders['val'])
torch.save(model.state_dict(),f'./saved_models/trained/ResNet_state_dict.pth')

In [ ]:
#model = utils.load_model('./saved_models/trained/ResNet_state_dict.pth',
 #                   p=p, pl_lightning=False, model=resnet.ResNet)


In [ ]:
model_res = resnet.ResNet(input_size=p_res['input_size'],output_size=p_res['output_size'],params=p_res)
model_res.load_state_dict(torch.load('./saved_models/trained/ResNet_state_dict.pth'))
model_ae = utils.load_model(path='./saved_models/trained/trained_ae.pth',p=p_ae,pl_lightning=False,model=SupAE)
models_dict = {'ResNet':[model_res,p_res],'ae':[model_ae,p_ae]}


In [ ]:
utils.create_predictions(models=models_dict)

In [ ]:
p['input_size'] = len(features)
p['output_size'] = 1
train = True
if train:
    gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
    models=[]
    for i, (train_idx,val_idx) in enumerate(gts.split(data,groups=era)):
        dataset = utils.FinData(data=data, target=target, era=era)
        dataloaders = utils.create_dataloaders(
        dataset, indexes={'train': train_idx, 'val': val_idx}, batch_size=p['batch_size'])
        model = SupAE(p)
        es = EarlyStopping(monitor='val_loss', patience=10,
                            min_delta=0.005, mode='min')
        trainer = pl.Trainer(max_epochs=100,
                                gpus=1,
                                callbacks=[es])
        trainer.fit(
            model, train_dataloader=dataloaders['train'], val_dataloaders=dataloaders['val'])
        torch.save(model.state_dict(), f'saved_models/ae_fold_{i}_state_dict.pth')
        models.append(model)
else:
    models_nn = utils.load_model('./saved_models/AE',p=p,pl_lightning=False,model=SupAE)


In [ ]:
data_,_,_,_ = utils.preprocess_data(df,nn=True)

p = {'learning_rate': 0.030803514080008098,
         'max_leaves': 50,
         'bagging_fraction': 0.9011886437667906,
         'bagging_freq': 5,
         'feature_fraction': 0.3287921216266973,
         'min_data_in_leaf': 396,
         'lambda_l1': 0.02217696438630042,
         'lambda_l2': 0.03985756503899372,
         'boosting': 'gbdt',
         'max_depth': 50,
         'objective': 'regression',
         'metric': 'mse',
         'n_jobs':-1}
gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
models_gbm = []
scores = []
for i, (tr_idx,val_idx) in enumerate(gts.split(data_,groups=era)):
    x_tr, x_val = data_[tr_idx], data_[val_idx]
    y_tr, y_val = target[tr_idx],target[val_idx]
    d_tr = lgb.Dataset(x_tr,label=y_tr)
    d_val = lgb.Dataset(x_val,label=y_val)
    clf = lgb.train(p,d_tr,1000,valid_sets=[d_val],early_stopping_rounds=50,verbose_eval=True)
    preds = clf.predict(x_val)
    score = mean_squared_error(y_val,preds)
    print(f'Fold {i} MSE score:\t', score)
    models_gbm.append(clf)
    scores.append(score)

In [ ]:
del data_,df,data

In [ ]:
input_size = data
output_size= 1
p = {'dim_1': 1500,
    'dim_2': 1000,
    'dim_3': 500,
    'dim_4': 250,
    'dim_5': 150,
    'activation': nn.SiLU,
    'dropout': 0.2,
    'lr': 0.05,
    'label_smoothing':0.1,
    'amsgrad': False,
    'batch_size': 5000,
    'loss':nn.MSELoss(),
    'embedding':True}

In [ ]:
dataset = utils.FinData(data=data, target=target, era=era)
del df

In [ ]:
model = models_nn[-1]
model.eval().to('cuda')
index = np.linspace(0,dataset.data.shape[0],dataset.data.shape[0],dtype='int')

In [ ]:
batch_size = 5000
dataloaders = utils.create_dataloaders(dataset,{'train':index.tolist()},batch_size=batch_size)

In [ ]:
hiddens =[]
for i, batch in enumerate(dataloaders['train']):
    _,hidden,_,_ = model(batch['data'].view(batch['data'].size(1),-1).to('cuda'))
    hiddens.append(hidden.cpu().detach().numpy())

In [ ]:
hiddens = np.array([hiddens[i][j] for i in range(len(hiddens)) for j in range(len(hiddens[i]))])

In [ ]:
data  = np.concatenate([data,hiddens],axis=1)

In [ ]:
gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
dataset = utils.FinData(data=data, target=target, era=era)
models=[]
for i, (train_idx,val_idx) in enumerate(gts.split(data,groups=era)):
    
    dataloaders = utils.create_dataloaders(
    dataset, indexes={'train': train_idx, 'val': val_idx}, batch_size=p['batch_size'])
    model = resnet.ResNet(input_size=input_size,output_size=output_size,params=p)
    #model.apply(lambda x: utils.init_weights(x,'relu'))
    es = EarlyStopping(monitor='val_mse', patience=10,
                        min_delta=0.005, mode='min')
    trainer = pl.Trainer(max_epochs=100,
                            gpus=1,
                            callbacks=[es])
    trainer.fit(
        model, train_dataloader=dataloaders['train'], val_dataloaders=dataloaders['val'])
    torch.save(model.state_dict(), f'saved_models/Resnet/fold_{i}_state_dict.pth')
    models.append(model)



In [ ]:
df = utils.load_data(root_dir='./data/',mode='test')

In [ ]:
data,target,features,era = utils.preprocess_data(df.iloc[:10000,],test=True,ordinal=True)
data_,_,_,_=utils.preprocess_data(df.iloc[:10000,],test=True,nn=True)
"""
data = data[0:1000]
oe = OrdinalEncoder()
data = oe.fit_transform(data)
"""

In [ ]:
t = torch.tensor()

In [ ]:
data = torch.LongTensor(data)

In [ ]:
data.shape

In [ ]:
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]


# convenience method for scoring
def scoring(df):
    return correlation(df['preds'], df['target'])


# Payout is just the score cliped at +/-25%
def payout(scores):
    return scores.clip(lower=-0.25, upper=0.25)

In [ ]:
preds = []
for model in models_nn:
    model.eval()
    preds.append(model(data))


In [ ]:
predictions = [preds[i][1].detach().numpy() for i in range(len(preds))]

In [ ]:
predictions = np.mean(predictions,axis=0)

In [ ]:
predictions = predictions.reshape(-1)

In [ ]:
df_preds = pd.DataFrame.from_dict({'era':era,'preds':predictions,'target':target.T})


In [ ]:
corrs = df_preds.groupby('era').apply(scoring)
corrs

In [ ]:
payout(corrs).mean()


In [ ]:
predictions = models_nn[0](data).reshape(-1).detach().numpy()

In [ ]:
preds = []
for model in models_gbm:
    preds.append(model.predict(data_))

In [ ]:
predictions_gbm = np.mean(preds,axis=0)
df_gbm = pd.DataFrame.from_dict({'era':era,'preds':predictions_gbm,'target':target.T})

In [ ]:
corrs_gbm = df_gbm.groupby('era').apply(scoring)
corrs_gbm

In [ ]:
payout(corrs_gbm).mean()

In [ ]:
df[df['data_type']=='test']

In [ ]:

wojhed-riDni0-hopnok